#### Lawand Yaseen 
##### Coding Discussion 5 
##### 11/14/2021

In [ ]:
#Goal 
#Build Naive Bayesian Classifer to predict whether respondent will vote or not pr(Vote ==1)

In [23]:
import pandas as pd
import numpy as np
import pprint as pp # for printing
import warnings
warnings.filterwarnings("ignore")

In [24]:
#set seed
np.random.seed(1234)

In [25]:
#read in data
turnout = pd.read_csv("/Users/lawandyaseen/Desktop/coding_discussions_ppol564_fall2021/05_coding_discussion/turnout.csv")

#discrete variables - white   
#continous - income, educate, age
turnout.head()

,id,age,educate,income,vote,white
0,1,60,14.0,3.3458,1,1
1,2,51,10.0,1.8561,0,1
2,3,24,12.0,0.6304,0,1
3,4,38,8.0,3.4183,1,1
4,5,25,12.0,2.7852,1,1


In [26]:
#Train-Test split

#80% training, 20% test 
train = turnout.sample(frac=.8).reset_index(drop=True)
test = turnout.drop(train.index).reset_index(drop=True)

#confimation of subset size
print("Training Data:", train.shape[0], "\n Test Data:", test.shape[0])

#check of training subset
train.head()

Training Data: 1600 
 Test Data: 400


,id,age,educate,income,vote,white
0,1749,78,16.0,1.3131,1,1
1,935,72,3.0,0.6765,1,0
2,1034,64,10.0,1.3131,1,1
3,244,80,8.0,1.1839,1,1
4,929,19,14.0,2.9072,1,1


In [27]:
#reshaping data continous and discrete data into quartiles based on values within each column
train['age_quartile'] = pd.qcut(train['age'], q=4, labels = ['25', '50', '75', '100'])
train['educate_quartile'] = pd.qcut(train['educate'], q=4, labels = ['25', '50', '75', '100'])
train['income_quartile'] = pd.qcut(train['income'], q=4, labels =  ['25', '50', '75', '100'])

#dropping old columns 
train = train.drop(columns = ['age', 'educate', 'income', 'id'])

train.head() #quartiles match expected values

,vote,white,age_quartile,educate_quartile,income_quartile
0,1,1,100,100,25
1,1,0,100,25,25
2,1,1,100,25,25
3,1,1,100,25,25
4,1,1,25,75,50


In [41]:
# calculating class probabilities 

N = train.shape[0] #number of observations in train

# Subset the data by class (voted or not voted)
vote1 = train.query("vote == 1")
vote0 = train.query("vote == 0")

# Calculate the probability for each class
pr_vote_1 = vote1.shape[0]/N
pr_vote_0 = vote0.shape[0]/N

In [35]:
# Vote = 1 subclass probabilities 

#done manually (couldn't get loop to work to automatically fill dictionary later on)

#age quartiles
age25_vote1 = vote1.query("age_quartile == '25'").shape[0]/vote1.shape[0]
age50_vote1 = vote1.query("age_quartile == '50'").shape[0]/vote1.shape[0]
age75_vote1 = vote1.query("age_quartile == '75'").shape[0]/vote1.shape[0]
age100_vote1 = vote1.query("age_quartile == '100'").shape[0]/vote1.shape[0]

#educate quartiles
educate25_vote1 = vote1.query("educate_quartile == '25'").shape[0]/vote1.shape[0]
educate50_vote1 = vote1.query("educate_quartile == '50'").shape[0]/vote1.shape[0]
educate75_vote1 = vote1.query("educate_quartile == '75'").shape[0]/vote1.shape[0]
educate100_vote1 = vote1.query("educate_quartile == '100'").shape[0]/vote1.shape[0]

#income quartiles 
income25_vote1 = vote1.query("income_quartile == '25'").shape[0]/vote1.shape[0]
income50_vote1 = vote1.query("income_quartile == '50'").shape[0]/vote1.shape[0]
income75_vote1 = vote1.query("income_quartile == '75'").shape[0]/vote1.shape[0]
income100_vote1 = vote1.query("income_quartile == '100'").shape[0]/vote1.shape[0]

#white 
white1_vote1 = vote1.query("white == 1").shape[0]/vote1.shape[0]
white0_vote1 = vote1.query("white == 0").shape[0]/vote1.shape[0]


#Vote = 0 subclass probabilities 

#age quartiles
age25_vote0 = vote0.query("age_quartile == '25'").shape[0]/vote1.shape[0]
age50_vote0 = vote0.query("age_quartile == '50'").shape[0]/vote1.shape[0]
age75_vote0 = vote0.query("age_quartile == '75'").shape[0]/vote1.shape[0]
age100_vote0 = vote0.query("age_quartile == '100'").shape[0]/vote1.shape[0]

#educate quartiles
educate25_vote0 = vote0.query("educate_quartile == '25'").shape[0]/vote1.shape[0]
educate50_vote0 = vote0.query("educate_quartile == '50'").shape[0]/vote1.shape[0]
educate75_vote0 = vote0.query("educate_quartile == '75'").shape[0]/vote1.shape[0]
educate100_vote0 = vote0.query("educate_quartile == '100'").shape[0]/vote1.shape[0]

#income quartiles 

income25_vote0 = vote0.query("income_quartile == '25'").shape[0]/vote1.shape[0]
income50_vote0 = vote0.query("income_quartile == '50'").shape[0]/vote1.shape[0]
income75_vote0 = vote0.query("income_quartile == '75'").shape[0]/vote1.shape[0]
income100_vote0 = vote0.query("income_quartile == '100'").shape[0]/vote1.shape[0]

#white 
white1_vote0 = vote0.query("white == 1").shape[0]/vote1.shape[0]
white0_vote0 = vote0.query("white == 0").shape[0]/vote1.shape[0]

In [42]:
#Empty dictionaries for classifier
class_probs = {};cond_probs = {}

#class probabilities 
class_probs[0] = pr_vote_0
class_probs[1] = pr_vote_1
       
#subset/conditional probabilities from previous chunk 
cond_probs['white',1,1] = white1_vote1
cond_probs['white',0,1] = white0_vote1
cond_probs['age_quartile','25',1] = age25_vote1
cond_probs['age_quartile','50',1] = age25_vote1
cond_probs['age_quartile','75',1] = age25_vote1
cond_probs['age_quartile','100',1] = age25_vote1
cond_probs['educate_quartile','25',1] = educate25_vote1
cond_probs['educate_quartile','50',1] = educate25_vote1
cond_probs['educate_quartile','75',1] = educate25_vote1
cond_probs['educate_quartile','100',1] = educate25_vote1
cond_probs['income_quartile','25',1] = income25_vote1
cond_probs['income_quartile','50',1] = income25_vote1
cond_probs['income_quartile','75',1] = income25_vote1
cond_probs['income_quartile','100',1] = income25_vote1
cond_probs['white',1,0] = white1_vote0
cond_probs['white',0,0] = white0_vote0
cond_probs['age_quartile','25',0] = age25_vote0
cond_probs['age_quartile','50',0] = age25_vote0
cond_probs['age_quartile','75',0] = age25_vote0
cond_probs['age_quartile','100',0] = age25_vote0
cond_probs['educate_quartile','25',0] = educate25_vote0
cond_probs['educate_quartile','50',0] = educate50_vote0
cond_probs['educate_quartile','75',0] = educate75_vote0
cond_probs['educate_quartile','100',0] = educate100_vote0
cond_probs['income_quartile','25',0] = income25_vote0
cond_probs['income_quartile','50',0] = income50_vote0
cond_probs['income_quartile','75',0] = income75_vote0
cond_probs['income_quartile','100',0] = income100_vote0
        
    
# Print for confirmation 
print("class probabilities",end="\n\n")
pp.pprint(class_probs)
print("\n")
print("conditional probabilities",end="\n\n")
pp.pprint(cond_probs)

class probabilities

{0: 0.2575, 1: 0.7425}


conditional probabilities

{('age_quartile', '100', 0): 0.12962962962962962,
 ('age_quartile', '100', 1): 0.20875420875420875,
 ('age_quartile', '25', 0): 0.12962962962962962,
 ('age_quartile', '25', 1): 0.20875420875420875,
 ('age_quartile', '50', 0): 0.12962962962962962,
 ('age_quartile', '50', 1): 0.20875420875420875,
 ('age_quartile', '75', 0): 0.12962962962962962,
 ('age_quartile', '75', 1): 0.20875420875420875,
 ('educate_quartile', '100', 0): 0.027777777777777776,
 ('educate_quartile', '100', 1): 0.20454545454545456,
 ('educate_quartile', '25', 0): 0.14225589225589225,
 ('educate_quartile', '25', 1): 0.20454545454545456,
 ('educate_quartile', '50', 0): 0.13383838383838384,
 ('educate_quartile', '50', 1): 0.20454545454545456,
 ('educate_quartile', '75', 0): 0.04292929292929293,
 ('educate_quartile', '75', 1): 0.20454545454545456,
 ('income_quartile', '100', 0): 0.03367003367003367,
 ('income_quartile', '100', 1): 0.19528619528619529,


In [37]:
#making class prediction based on conditional probabilities for each observation 

#compares probability of two classes based off observational data for each individual and assigns 1 or 0
#based off which probability is larger (more likely occurence)

def predict(data,class_probs,cond_probs):
    '''
    Function calculates the conditiona probability for membership into each class.
    Then returns both the probabilities and the most likely class. 
    '''
    #empty list where prediction will go 
    
    store_preds = []
    for i,row in data.iterrows():
        pr_1 = 1; pr_0 = 1
        for j in range(1,len(row.index)):
            pr_0 *= cond_probs[(row.index[j],row.values[j],0)]
            pr_1 *= cond_probs[(row.index[j],row.values[j],1)]     
        pr_0 *= class_probs[0]
        pr_1 *= class_probs[1]
        #adding 1 if pr_1 is greater than pr_0 
        store_preds.append([pr_0,pr_1,max([(pr_0,0),(pr_1,1)])[1]])
    return pd.DataFrame(store_preds,columns=["pr_0","pr_1","pred"])

#Running prediction with train data
preds = predict(train, class_probs, cond_probs)
preds.head()

,pr_0,pr_1,pred
0,0.000037,0.005378,1
1,0.000047,0.000813,1
2,0.000187,0.005378,1
3,0.000187,0.005378,1
4,0.000038,0.005378,1


In [38]:
#predicting accuracy with training data 
accuracy = sum(train.vote == preds.pred)/train.shape[0]
accuracy

0.7425

In [39]:
#checking accuract with test data 

#reshaping data to follow quartile format used for training data 
test['age_quartile'] = pd.qcut(test['age'], q=4, labels = ['25', '50', '75', '100'])
test['educate_quartile'] = pd.qcut(test['educate'], q=4, labels = ['25', '50', '75', '100'])
test['income_quartile'] = pd.qcut(test['income'], q=4, labels =  ['25', '50', '75', '100'])

#dropping columns 
test = test.drop(columns = ['age', 'educate', 'income', 'id'])

KeyError: 'age'

In [40]:
#running test data predictions 
test_preds = predict(test, class_probs, cond_probs)
test_accuracy = sum(test.vote == test_preds.pred)/test.shape[0]
test_accuracy

0.7

Using the classifer returned around 75% accuracy for the training data and an accuracy of 70% when using the test data. This result shows that the model is better than a coin flip when taking into account the reported data regarding each individual. 